In [4]:
import pandas as pd
import matplotlib.pyplot as plt

#### Загрузить данные в pandas по url=https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv

#### 1. Вывести: кол-во наблюдений в датасете
#### 2. Вывести названия столбцов
#### 3. Определить самую частую позицию (item) в заказе 
#### 4. Построить гистрограмму частоты заказов по позициям (item )
#### 5. Измените тип переменной item_price c с помощью лямбды функции
#### 6. Построить гистограмму кол-во денег заработанных по каждой позиции (item)
#### 7. Средняя сумма заказа? (минимум 2 способа)
#### 8. Выведите среднее, минимальное и максимальное, медианное значения позиций в заказе
#### 9. Определить статистику заказов стейков, а также статистику заказов прожарки.
#### 10. Добавить новый столбец цен на каждую позицию в заказе в рублях.
#### 11. Сгруппировать заказы по входящим позициям в него. Отдельно сгруппировать по стейкам во всех видах прожарках.
#### 12. Определить цену по каждой позиции в отдельности. 


In [24]:
data = pd.read_csv("https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv", sep='\t')

In [ ]:
data.head()

In [85]:
# 1 Вывести кол-во наблюдений в датасете
print(len(data))

4622


In [84]:
# 2 Вывести названия столбцов
print(*data.columns, sep='\n')

order_id
quantity
item_name
choice_description
item_price


In [64]:
# 3 Определить самую частую позицию в заказе
print(data['item_name'].mode()[0])

Chicken Bowl


In [ ]:
# 5 Измените тип переменной item_price c с помощью лямбда функции
data['item_price'] = data['item_price'].apply(lambda x: float(x[1:]))

In [81]:
# 7 Средняя сумма заказа
# first variant
orders_number = len(data['order_id'].value_counts())
print((data['item_price'] * data['quantity']).sum() / orders_number)

21.39423118865867


In [60]:
# 8 Выведите среднее, минимальное и максимальное, медианное значения позиций в заказе
orders_data = data.groupby('order_id').size()
print('average:', orders_data.mean())
print('min:', orders_data.min())
print('max:', orders_data.max())
print('mode:', orders_data.mode()[0])

average: 2.520174482006543
min: 1
max: 23
mode: 2


### Алгоритмы уменьшения размерности 

#### повторить эксперименты в главе 5.1 Qalitative Comparison of Multiple Algorithms статьи статьи UMAP: Uniform Manifold
#### Approximation and Projection for Dimension Reduction (https://arxiv.org/pdf/1802.03426.pdf)

#### необходимо воспроизвести все графики данной главы, соотнести выводы из статьи с вашими результатами.